In [1]:
def hint(string): return print(''.join(map(chr, map(lambda x: int(x, 16), string.split('.')))))

## 第一部分 背景与问题描述

## Environment
+ jupyter notebook (测试、试验环境)
+ Pycharm （开发环境）
+ python3.6
+ networkx
+ jieba
+ numpy, pandas, matplotlib
+ gensim
+ (optional) bottle

## Background

在自然语言处理中，有一个常见的问题就是对客户的评价进行分析。 这些用户评论中，包含了大量的有用信息，例如情感分析，或者相关事实描述。 例如，
> `“味道不错的面馆，性价比也相当之高，分量很足～女生吃小份，胃口小的，可能吃不完呢。环境在面馆来说算是好的，至少看上去堂子很亮，也比较干净，一般苍蝇馆子还是比不上这个卫生状况的。中午饭点的时候，人很多，人行道上也是要坐满的，隔壁的冒菜馆子，据说是一家，有时候也会开放出来坐吃面的人。“`

首先情感是正向的，除此之外我们还能够进行知道这个的几个事实描述：1. 性价比比较高； 2. 装修比较好； 3. 分量足。 

这些信息是非常重要宝贵的，不论是对于公司进行商业分析或者要建立一个搜索引擎排序，这些信息都是重要的参考因素。 那么在这个时候，我们就需要进行文本的情感分类了。

**注**: 此次问题的来源数据集来源于大众点评， 这个数据集也被用作 AI 全球挑战赛的数据集。 而且细粒度情感分类这个问题其实在目前而言是一个 *未解之谜*， 人类现在对这个问题并没有什么特别好的方法， 因为人的情感表达真的是很有变化的，例如“我不认为这个地方不好是不对的”。所以这个问题也需要大家在之后做出来基础模型之后，大家多想想办法，*八仙过海，各显神通*

所以，我们这一次是要使用深度学习的方法，建立一个模型，这个模型能够将一句话进行分类判断，判断出来这句话到底表达了什么重要信息。 说实话，这个看似简单的问题，曾经是困扰了科学家数十年的问题，就算是现在，深度学习，人工智能有了很大的进步，其效果也达不到人们预期的那么好,但是比起前些年已经好多了:)

这个问题我们希望的是，输入一句话，输出是这句话对于以下6大类，20小类进行打标，对于每个小类而言，都会有<** 正面情感, 中性情感, 负面情感, 情感倾向未提及 > ** 这4个类别。 

总得来说，我们现在这6大类，20小类的类别如下：

+ 位置: location
    + 交通是否便利(traffic convenience)
    + 距离商圈远近(distance from business district)
    + 是否容易寻找(easy to find)
+ 服务(service)	
    + 排队等候时间(wait time)
    + 服务人员态度(waiter’s attitude)
    + 是否容易停车(parking convenience)
    + 点菜/上菜速度(serving speed)
+ 价格(price)	
    + 价格水平(price level)
    + 性价比(cost-effective)
    + 折扣力度(discount)
+ 环境(environment)	
    + 装修情况(decoration)
    + 嘈杂情况(noise)
    + 就餐空间(space)
    + 卫生情况(cleaness)
+ 菜品(dish)	
    + 分量(portion)
    + 口感(taste)
    + 外观(look)
    + 推荐程度(recommendation)
+ 其他(others)	
    + 本次消费感受(overall experience)
    + 再次消费的意愿(willing to consume again)

而为了方便训练数据的标标注，训练数据中，<** 正面情感, 中性情感, 负面情感, 情感倾向未提及 > ** 分别对应与 (1, 0, -1, -2). 

例如说，
> `“味道不错的面馆，性价比也相当之高，分量很足～女生吃小份，胃口小的，可能吃不完呢。环境在面馆来说算是好的，至少看上去堂子很亮，也比较干净，一般苍蝇馆子还是比不上这个卫生状况的。中午饭点的时候，人很多，人行道上也是要坐满的，隔壁的冒菜馆子，据说是一家，有时候也会开放出来坐吃面的人。“`

这句话在训练数据中的标签就是：

+ 交通是否便利(traffic convenience)	-2 
+ 距离商圈远近(distance from business district)	-2
+ 是否容易寻找(easy to find)	-2
+ 排队等候时间(wait time)	-2
+ 服务人员态度(waiter’s attitude)	-2
+ 是否容易停车(parking convenience)	-2
+ 点菜/上菜速度(serving speed)	-2
+ 价格水平(price level)	-2
+ 性价比(cost-effective)	1
+ 折扣力度(discount)	-2
+ 装修情况(decoration)	1
+ 嘈杂情况(noise)	-2
+ 就餐空间(space)	-2
+ 卫生情况(cleaness)	1
+ 分量(portion)	1
+ 口感(taste)	1
+ 外观(look)	-2
+ 推荐程度(recommendation)	-2
+ 次消费感受(overall experience)	1
+ 再次消费的意愿(willing to consume again)	-2

## 数据集下载

数据集的下载在 https://challenger.ai/competition/fsauor2018， 大家下载数据集， 以及测试集，注意，训练集我们需要分成 training data, validation data 然后 test data里边的数据绝对不能在训练的时候用。 否则的话就是去了意义。 

**注**, 这个数据集之所以被用到了 AI 挑战赛中，是因为其难度很大。 绝大数人在公司中遇到的问题难度**不会**超过这个问题。 

## 评价

参照 https://challenger.ai/competition/fsauor2018, 这个问题的评价其实就是多个分类的f1 score 的平均值。

## 第二部分 基础理论部分

#### Q1: 机器学习中的Loss函数的作用为何？

回答：

In [3]:
# remove the # before hint, what you find? 
hint('8fd9.4e2a.7b54.6848.53ef.4ee5.5199.5f88.591a.ff0c.4f46.662f.4e3b.8981.662f.8981.6d89.53ca.5230.ff1a.4c.6f.73.73.20.51fd.6570.7528.6765.8861.91cf.673a.5668.5b66.4e60.8fc7.7a0b.4e2d.6a21.578b.8868.73b0.7684.ff0c.6211.4eec.901a.8fc7.20.4c.6f.73.73.20.51fd.6570.6765.8fdb.884c.4f18.5316.6a21.578b.6216.8005.9009.62e9.6a21.578b')

这个答案可以写很多，但是主要是要涉及到：Loss 函数用来衡量机器学习过程中模型表现的，我们通过 Loss 函数来进行优化模型或者选择模型


####  Q2: 为什么 SVM 适合核函数的方法？（考虑基于拉格朗日距离的 SVM 的 Loss 函数）

回答：

In [4]:
hint('8be6.60c5.8bf7.53c2.8003.6211.4eec.7684.8bfe.7a0b.89c6.9891.ff0c.4e00.4e2a.4e3b.8981.7684.70b9.662f.ff0c.6700.540e.8bc1.660e.20.53.56.4d.20.6a21.578b.7684.6027.80fd.53ea.4e0e.20.78.5f.69.78.5f.6a.20.7684.4e58.673a.76f8.5173.ff0c.6240.4ee5.6211.4eec.53ef.4ee5.65b9.4fbf.7684.628a.78.5f.69.20.78.5f.6a.20.6620.5c04.5230.67d0.4e2a.65b0.51fd.6570.4e0a.ff0c.4e0d.6539.53d8.5176.20.4c.6f.73.73.20.7684.5355.8c03.6027.5373.53ef')

详情请参考我们的课程视频，一个主要的点是，最后证明 SVM 模型的性能只与 x_ix_j 的乘机相关，所以我们可以方便的把x_i x_j 映射到某个新函数上，不改变其 Loss 的单调性即可


#### Q3: 决策树的 Loss 函数是什么？随机森林是什么？ 

回答:

In [5]:
hint("""4e00.3001.71b5.7684.548c.6700.5c0f.ff0c.8981.9009.62e9.4e00.4e2a.6761.4ef6.8ba9.8fd9.6b21.5206.7c7b.7684.4e24.8fb9.7ed3.679c.7adf.53ef.80fd.7684.2018.7eaf.2019.2c.20.4f8b.5982.ff0c.6211.4eec.6709.5b.32.2c.20.31.2c.20.30.2c.20.30.2c.20.30.2c.20.30.2c.20.31.2c.20.31.2c.20.31.2c.20.31.2c.20.31.5d.a.5982.679c.6211.4eec.9009.62e9.4e00.4e2a.6761.4ef6.662f.27.662f.4e0d.662f.31.27.ff0c.6211.4eec.53ef.4ee5.5206.6210.5b.30.2c.20.30.2c.20.30.2c.20.30.2c.20.32.5d.2c.20.5b.31.2c.20.31.2c.20.31.2c.20.31.2c.20.31.5d.ff0c.20.4e5f.53ef.4ee5.6761.4ef6.662f.27.662f.4e0d.662f.32.27.2c.20.a.6211.4eec.5c31.53ef.4ee5.5206.6210.5b.32.5d.2c.20.5b.31.2c.20.30.2c.20.30.2c.20.30.2c.20.30.2c.20.31.2c.20.31.2c.20.31.2c.20.31.2c.20.31.5d.ff0c.20.663e.7136.524d.8005.66f4.7eaf.3002.20.5982.679c.540c.5b66.4eec.5fd8.4e86.71b5.7684.6982.5ff5.ff0c.5927.5bb6.8d76.7d27.518d.67e5.4e00.4e0b.ff0c.7136.540e.8ba1.7b97.4e00.4e0b.8fd9.4e24.4e2a.5206.7c7b.7ed3.679c.7684.71b5.662f.591a.5927.ff1b.a.4e8c.3001.968f.673a.68ee.6797.662f.7528.5f88.591a.5c0f.7684.51b3.7b56.6811.7528.6765.6295.7968.7684.96c6.6210.6a21.578b.ff08.45.6e.73.65.6d.62.6c.65.ff09.ff0c.6bcf.4e2a.5c0f.51b3.7b56.6811.4f7f.7528.4e00.90e8.5206.7684.20.66.65.61.74.75.72.65.20.8fdb.884c.8bad.7ec3""")

一、熵的和最小，要选择一个条件让这次分类的两边结果竟可能的‘纯’, 例如，我们有[2, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1]
如果我们选择一个条件是'是不是1'，我们可以分成[0, 0, 0, 0, 2], [1, 1, 1, 1, 1]， 也可以条件是'是不是2', 
我们就可以分成[2], [1, 0, 0, 0, 0, 1, 1, 1, 1, 1]， 显然前者更纯。 如果同学们忘了熵的概念，大家赶紧再查一下，然后计算一下这两个分类结果的熵是多大；
二、随机森林是用很多小的决策树用来投票的集成模型（Ensemble），每个小决策树使用一部分的 feature 进行训练


**插入**：我们在这里快速的过一下如何在 Jupyter 中写数学公式。 这个其实很简单，如果我们要输入一个公式，例如A1, 那么，我们在Jupyter中输入`$A_i$`, 然后 Enter， 是不是就变成了$A_i$? 其实两个`$$`之间的东西就是 Latex 的符号，`$..$`这个我们叫做inline模式，意思就是说你写出来的公式是和你的文字在一行里，如果你`$$..$$``，这个公式就会单独是一行。

我们现在再试一个, 输入`$$\frac{P_i}{\sum_{j \in \mathbf{V}}^NP_j}$$`, 输完之后 Enter， 你看到了什么？ 

$$\frac{P_i}{\sum_{j \in \mathbf{V}}^NP_j}$$

这个时候会有同学说，可是这些符号，我怎么记得住呢？ 我给大家提供了一个参考手册，大家有空就看看 https://github.com/Artificial-Intelligence-for-NLP/comment-setimental-classification/blob/master/Latex-Symbols.pdf，熟能生巧。 


#### Q4: 使用Latex 写出来决策树希望找到一个 feature，这个 feature 使得熵的和最少的公式。

回答：

##### Q5: 贝叶斯公式的原理是什么？ 我们现在用的贝叶斯分类器为什么是“朴素贝叶斯”， 它为什么朴素？

回答：

#### Q6: 神经网络的Loss函数的作用为何？

回答：

In [6]:
hint("""795e.7ecf.7f51.7edc.91cc.9762.7684.4c.6f.73.73.51fd.6570.662f.7528.6765.8861.91cf.6a21.578b.7684.597d.574f.ff0c.4c.6f.73.73.51fd.6570.8d8a.5927.ff0c.9884.6d4b.4e0e.5b9e.9645.7684.8bef.5dee.8d8a.5927.ff0c.9884.6d4b.8d8a.4e0d.51c6.786e.3002.4e3a.4e86.8ba9.9884.6d4b.7ed3.679c.66f4.52a0.7cbe.51c6.ff0c.6211.4eec.8981.51cf.5c11.4c.6f.73.73.51fd.6570.ff0c.901a.8fc7.68af.5ea6.4e0b.964d.6cd5.ff0c.5229.7528.53cd.5411.4f20.64ad.4e0d.505c.8fed.4ee3.8c03.6574.795e.7ecf.7f51.7edc.4e2d.7684.53c2.6570.ff0c.627e.5230.4f7f.4c.6f.73.73.51fd.6570.6700.5c0f.7684.53c2.6570.ff0c.786e.5b9a.6a21.578b.3002""")

神经网络里面的Loss函数是用来衡量模型的好坏，Loss函数越大，预测与实际的误差越大，预测越不准确。为了让预测结果更加精准，我们要减少Loss函数，通过梯度下降法，利用反向传播不停迭代调整神经网络中的参数，找到使Loss函数最小的参数，确定模型。


#### Q7: 神经网络的激活函数(activation function)起什么作用？ 如果没有激活函数会怎么样？ 

回答：

In [7]:
hint('6fc0.6d3b.51fd.6570.7528.6765.8fdb.884c.975e.7ebf.6027.53d8.5316.ff0c.4e0d.65ad.5f97.975e.7ebf.6027.53d8.5316.4f7f.5f97.6211.4eec.28.7406.8bba.4e0a.29.53ef.4ee5.62df.5408.4efb.610f.51fd.6570.ff0c.8fd9.4e5f.662f.4e3a.4ec0.4e48.795e.7ecf.7f51.7edc.80fd.60.5b66.4e60.60.7684.539f.56e0.3002.795e.7ecf.7f51.7edc.91cc.8fb9.7684.60.5b66.4e60.60.5176.5b9e.5c31.662f.51fd.6570.62df.5408.7684.610f.601d')

激活函数用来进行非线性变化，不断得非线性变化使得我们(理论上)可以拟合任意函数，这也是为什么神经网络能`学习`的原因。神经网络里边的`学习`其实就是函数拟合的意思


#### Q8: 神经网络的softmax如何理解， 其作用是什么？ 在`答案`中写出softmax的python表达；


回答：

#### Q9: 简述 normalized_1 和softmax函数的相同点和不同点， 说明softmax相比normalized_1该函数的优势所在

```
output = np.array([y1, y2, y3])

normalized_1 = output / np.sum(output)
```

回答：

#### Q10: 写出crossentropy的函数表达式，说明该函数的作用和意义

回答：

..

..

----------------- 休息一下，接下来是关于 Word2Vec的 ------------- 

#### Q11: 说明word2vec要解决的问题背景， 以及word2vec的基本思路， 说明word2vec比起之前方法的优势；

回答：

#### Q12: 说明word2vec的预测目标， predication target, 在答案中写出skip-gram和cbow的预测概率；

回答：

#### Q13: 请说明word2vec的两种常见优化方法，分别阐述其原理；

回答：

#### Q14: 请说明word2vec中哈夫曼树的作用；

回答：

#### Q15: 哈夫曼树如何构建？

In [8]:
hint('a.31.2e.20.68.74.74.70.73.3a.2f.2f.67.69.74.68.75.62.2e.63.6f.6d.2f.68.65.69.6e.65.6d.61.6e.2f.70.79.74.68.6f.6e.2d.64.61.74.61.2d.73.74.72.75.63.74.75.72.65.73.2f.62.6c.6f.62.2f.6d.61.73.74.65.72.2f.35.2e.25.32.30.48.65.61.70.2d.62.61.73.65.64.25.32.30.53.74.72.75.63.74.75.72.65.73.2f.68.75.66.66.6d.61.6e.2e.70.79.a.32.2e.20.68.74.74.70.73.3a.2f.2f.67.69.74.68.75.62.2e.63.6f.6d.2f.52.61.52.65.2d.54.65.63.68.6e.6f.6c.6f.67.69.65.73.2f.67.65.6e.73.69.6d.2f.62.6c.6f.62.2f.33.64.35.61.32.31.63.31.63.38.31.32.38.63.62.38.64.64.34.66.36.65.35.31.65.39.65.66.33.64.63.35.61.66.30.30.30.38.37.31.2f.67.65.6e.73.69.6d.2f.6d.6f.64.65.6c.73.2f.64.65.70.72.65.63.61.74.65.64.2f.77.6f.72.64.32.76.65.63.2e.70.79.23.4c.36.37.30.22.a.33.2e.20.68.74.74.70.73.3a.2f.2f.77.77.77.2e.77.69.6b.69.77.61.6e.64.2e.63.6f.6d.2f.65.6e.2f.48.75.66.66.6d.61.6e.5f.63.6f.64.69.6e.67.a')


1. https://github.com/heineman/python-data-structures/blob/master/5.%20Heap-based%20Structures/huffman.py
2. https://github.com/RaRe-Technologies/gensim/blob/3d5a21c1c8128cb8dd4f6e51e9ef3dc5af000871/gensim/models/deprecated/word2vec.py#L670"
3. https://www.wikiwand.com/en/Huffman_coding



#### Q16: 在gensim中如何实现词向量？ 请将gensim中实现词向量的代码置于答案中

回答：

#### Q17: 请说出除了 skip-gram和cbow的其他4中词向量方法的名字， 并且选取其中两个叙述其基本原理。

回答：

In [9]:
hint('a.4f.6e.65.68.6f.74.2c.20.47.6c.6f.76.65.2c.43.6f.76.65.2c.45.4d.4c.6f.a')


Onehot, Glove,Cove,EMLo



-------------------- 休息一下，接下来是关于 Keras 和 Tensorflow 使用的 -------------- 

大家先熟悉一下什么是MNIST数据集： 

> http://yann.lecun.com/exdb/mnist/

>https://en.wikipedia.org/wiki/MNIST_database

#### Q18: 参考keras参考手册，构建一个机器学习模型，该模型能够完成使用DNN(deep neural networks) 实现MNIST数据集的分类；

关键代码: 

#### Q19:参考tensorflow的参考手册，构建一个机器学习模型，该模型能够完成使用DNN(deep neural networks)实现MNIST数据集的分类；

关键代码：

In [10]:
hint('68.69.6e.74.73.3a.74.65.6e.73.6f.72.66.6c.6f.77.5b9e.73b0.4d.4e.49.53.54.20.68.74.74.70.73.3a.2f.2f.67.69.74.68.75.62.2e.63.6f.6d.2f.74.65.6e.73.6f.72.66.6c.6f.77.2f.74.65.6e.73.6f.72.66.6c.6f.77.2f.62.6c.6f.62.2f.6d.61.73.74.65.72.2f.74.65.6e.73.6f.72.66.6c.6f.77.2f.65.78.61.6d.70.6c.65.73.2f.75.64.61.63.69.74.79.2f.32.5f.66.75.6c.6c.79.63.6f.6e.6e.65.63.74.65.64.2e.69.70.79.6e.62')

hints:tensorflow实现MNIST https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/2_fullyconnected.ipynb


#### Q20: 参考keras和tensorflow对同一问题的实现，说明keras和tensorflow的异同；

回答：

#### Q21: tensorflow 使用 Graph 计算机制的优缺点是什么？ 

回答：

#### Q22: Q18， Q19 的tensorflow 或 keras 模型的训练时准确率和测试集准确率分别是多少？

回答：

In [41]:
#### Q23: 训练时准确率大于测试集准确率的现象叫什么名字，在神经网络中如何解决该问题？(至少提出5个解决方法)

回答：

In [42]:
#### Q24: 请使用自己的语言简述通过正则化 (regularization)减小过拟合的原理；

回答：

In [43]:
#### Q25: 在tensorflow官方实例中给出的fully connected 神经网络的分类模型中，数据进行了哪些预处理，这些预处理的原因是什么？

回答：

--------------------- 休息一下，接下来是关于 RNN 和 CNN 的 ---------- 

#### Q26: 简述CNN的原理

回答：

#### Q27: CNN的 Spatial Invariant是什么意思？ 是如何做到的？ 

回答：

#### Q28: CNN增加了很多层数，这些层数使用 filter 进行计算。 按说需要拟合的参数变得很多，请问 CNN 是如何解决这个问题的，如何加快速度的？ 

回答：

In [11]:
hint('a.6d.61.69.6e.20.70.6f.69.6e.74.73.3a.20.50.6f.6f.6c.69.6e.67.2c.20.50.61.72.61.6d.65.74.65.72.20.53.68.61.72.69.6e.67.a')


main points: Pooling, Parameter Sharing



#### Q29: CNN中的 Batch Normalization有什么意义？

回答:

#### Q30: CNN中的 Pooling 起到什么作用？

回答：

#### Q31: CNN中的 Fully Connect起到什么作业？

回答：

#### Q32: 深度网络中的权值初始化有什么讲究？

回答：

#### Reading: 参照 Keras 和 Tensorflow 的示例，手敲使用 keras, tensorflow + CNN 实现MNIST分类的问题：

+ https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
+ https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/4_convolutions.ipynb

把代码手敲一遍

#### Q33: 简述RNN解决的问题所具有的特点；

回答：

#### Q34: 写出RNN实现时间或者序列相关的数学实现(见课程slides)

回答：

#### Q35: 简述RNN的两种重要变体的提出原因和基本原理？

回答: 

#### Q36:  Attentional RNN 以及 Stacked RNN 和 Bi-RNN 分别是什么，其做了什么改动？

回答：

#### Reading 和 CNN 类似，请在 Keras,Tensorflow中查找如何实现 RNN 模型

+ https://github.com/Artificial-Intelligence-for-NLP/References/blob/master/AI%20%26%20Machine%20Learning/Hands.On.TensorFlow.pdf

## 第二部分： 项目解决过程

代码主要在 Pycharm 里边写，jupyter 里边写一个关键步骤就行

#### Q37: 要实现文本分类或情感分类，文本信息需要进行哪些初始化操作？自己手工实现，keras提供的API，tenorflow提供的API，分别是哪些？请提供关键代码置于下边`回答`中

回答：

In [13]:
#hint('id_to_word, word_to_id, padding, batched')

#### Q38 在没有预训练的词向量时候， keras 如何实现embedding操作，即如何依据一个单词的序列获得其向量表示？

回答：

#### Q 39: 在**有**预先训练的词向量时候，keras和tensorflow又如何实现embeding操作

回答：

#### Q40：基于上文进行的数据预处理，使用keras和tensorflow如何构建神经网络模型？请提供关键代码

### 好的 现在开始切入正题 --

其实，我们解决实际问题的时候，很少自己从头到尾写一个神经网络模型，我们往往是找一个效果比较好的类似问题的模型，然后在这个问题上改造。 或者我们在去一个公司的时候，接手的工作也往往是改动以前的模型，所以我们解决这个语义分类问题我们也首先是找一个类似的问题，然后参考一个模型进行修改，变成能够解决我们这个问题的模型。

我们以上所以的理论知识，都是用来支持我们做修改，能够看懂别人为何要这样写，然后自己要改哪里。 

kaggle上的“恶意评价识别”这个项目和我们的这个项目是类似的, 大家请首先在这个的 Kernel 里边找到一个公开代码的示例，然后选择一个自己能够看懂且效果较好的模型进行改造。

+ https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge
+ https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/kernels


Kaggle这个问题和我们的问题类似，但是并不是完全一样， 其中最不一样的其实是我们的期望的结果这里, Kaggle 这里的输出是5个类别，然后类别的是0~1直接的数字来预测是否是这个类别，然后我们的客户评价问题中，打标对20个分类的-2, -1, 0, 1四个标记. 一种最简单的方法，是把这个20分类问题变成80分类问题，然后每个分类的输出是0或者1. 

### My solution

整体是借鉴的当时的参赛大佬的，当时他的f1_sroce是0.70201. 他的github:  https://github.com/pengshuang/AI-Comp .   
他用的是对每个类分别建模，用20个模型， 每个模型训练20epoch，确实取得了不错的效果但是即使是在google colab上运行，一个epoch还是要5分多钟，训练完20个模型花费的时间实在太长。 所以就想在此基础上改成80分类。此外用fasttext的pre-trained的词嵌入（原作者没有用pre-trained）。 看看是否对最终结果有所提升。

由于硬件限制，代码都放在googel colab上运行。

+ Preprocess

In [ ]:
#Feature engine
import random

random.seed = 16
import pandas as pd
from gensim.models.word2vec import Word2Vec

data = pd.read_csv("./input/sentiment_analysis_trainingset.csv")

stopwords = []
with open("../my_solution/input/stop_words.txt", encoding='gbk') as f:
    for line in f.readlines():
        line = line.strip()
        stopwords.append(line)


def segWord(doc):
    seg_list = jieba.cut(doc, cut_all=False)
    return list(seg_list)


# move stop words
def filter_map(arr):
    res = ""
    for c in arr:
        if c not in stopwords and c != ' ' and c != '\xa0' and c != '\n' and c != '\ufeff' and c != '\r':
            res += c
    return res


# move stop words and generate char sent
def filter_char_map(arr):
    res = []
    for c in arr:
        if c not in stopwords and c != ' ' and c != '\xa0' and c != '\n' and c != '\ufeff' and c != '\r':
            res.append(c)
    return " ".join(res)


# get char of sentence
def get_char(arr):
    res = []
    for c in arr:
        res.append(c)
    return list(res)
#labels ont-hot encoding, get 80 labels
def enconding_labels(datasets):
    cut_regions = [-3, -2, -1, 0, 1]
    labels = datasets.iloc[:, 2:]
    labels_cut_regions = {}

    for label in labels:
        labels_cut_regions[label] = cut_regions

    cut_df = pd.DataFrame()
    for field in labels_cut_regions.keys():
        cut_series = pd.cut(datasets[field], labels_cut_regions[field], right=True)
        onehot_df = pd.get_dummies(cut_series, prefix=field)
        cut_df = pd.concat([cut_df, onehot_df], axis=1)
    new_df = pd.concat([datasets, cut_df], axis=1)
    return new_df



data.content = data.content.map(lambda x: filter_map(x))
data.content = data.content.map(lambda x: get_char(x))
data = enconding_labels(data)
data.to_csv("preprocess/train_char.csv", index=None)

validation = pd.read_csv("./input/sentiment_analysis_validationset.csv")
validation.content = validation.content.map(lambda x: filter_map(x))
validation.content = validation.content.map(lambda x: get_char(x))
validation = enconding_labels(validation)
validation.to_csv("preprocess/validation_char.csv", index=None)

test = pd.read_csv("./input/sentiment_analysis_testa.csv")
test.content = test.content.map(lambda x: filter_map(x))
test.content = test.content.map(lambda x: get_char(x))
test = enconding_labels(test)
test.to_csv("preprocess/test_char.csv", index=None)

+ Join attention mechanism

In [ ]:
# coding=utf8
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.layers.merge import _Merge


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        # self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        input_shape = K.int_shape(x)

        features_dim = self.features_dim
        # step_dim = self.step_dim
        step_dim = input_shape[1]

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b[:input_shape[1]]

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    	# print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        # return input_shape[0], input_shape[-1]
    	return input_shape[0], self.features_dim
# end Attention


class JoinAttention(_Merge):
    def __init__(self, step_dim, hid_size,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism according to other vector.
        Supports Masking.
        # Input shape, list of
            2D tensor with shape: `(samples, features_1)`.
            3D tensor with shape: `(samples, steps, features_2)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            en = LSTM(64, return_sequences=False)(input)
            de = LSTM(64, return_sequences=True)(input2)
            output = JoinAttention(64, 20)([en, de])
        """
        self.supports_masking = True
        # self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.hid_size = hid_size
        super(JoinAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        if not isinstance(input_shape, list):
            raise ValueError('A merge layer [JoinAttention] should be called '
                             'on a list of inputs.')
        if len(input_shape) != 2:
            raise ValueError('A merge layer [JoinAttention] should be called '
                             'on a list of 2 inputs. '
                             'Got ' + str(len(input_shape)) + ' inputs.')
        if len(input_shape[0]) != 2 or len(input_shape[1]) != 3:
            raise ValueError('A merge layer [JoinAttention] should be called '
                             'on a list of 2 inputs with first ndim 2 and second one ndim 3. '
                             'Got ' + str(len(input_shape)) + ' inputs.')

        self.W_en1 = self.add_weight((input_shape[0][-1], self.hid_size),
                                 initializer=self.init,
                                 name='{}_W0'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.W_en2 = self.add_weight((input_shape[1][-1], self.hid_size),
                                 initializer=self.init,
                                 name='{}_W1'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.W_de = self.add_weight((self.hid_size,),
                                 initializer=self.init,
                                 name='{}_W2'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)

        if self.bias:
            self.b_en1 = self.add_weight((self.hid_size,),
                                     initializer='zero',
                                     name='{}_b0'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
            self.b_en2 = self.add_weight((self.hid_size,),
                                     initializer='zero',
                                     name='{}_b1'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
            self.b_de = self.add_weight((input_shape[1][1],),
                                     initializer='zero',
                                     name='{}_b2'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b_en1 = None
            self.b_en2 = None
            self.b_de = None

        self._reshape_required = False
        self.built = True

    def compute_output_shape(self, input_shape):
        return input_shape[1][0], input_shape[1][-1]

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, inputs, mask=None):
        en = inputs[0]
        de = inputs[1]
        de_shape = K.int_shape(de)
        step_dim = de_shape[1]

        hid_en = K.dot(en, self.W_en1)
        hid_de = K.dot(de, self.W_en2)
        if self.bias:
            hid_en += self.b_en1
            hid_de += self.b_en2
        hid = K.tanh(K.expand_dims(hid_en, axis=1) + hid_de)
        eij = K.reshape(K.dot(hid, K.reshape(self.W_de, (self.hid_size, 1))), (-1, step_dim))
        if self.bias:
            eij += self.b_de[:step_dim]

        a = K.exp(eij - K.max(eij, axis=-1, keepdims=True))

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask[1], K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = de * a
        return K.sum(weighted_input, axis=1)
# end JoinAttention


+ Model construction

In [ ]:
import keras
from keras import Model
from keras.layers import *
from JoinAttLayer import Attention     #using attention mechanism


def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred, beta=1):
    """Computes the F score.
    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.
    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


#line 7-53 define three evalution index.



class TextClassifier():                                       #model construction.

    def model(self, embeddings_matrix, maxlen, word_index, num_class):
        inp = Input(shape=(maxlen,))
        encode = Bidirectional(CuDNNGRU(128, return_sequences=True))
        encode2 = Bidirectional(CuDNNGRU(128, return_sequences=True))
        attention = Attention(maxlen)
        x_4 = Embedding(len(word_index) + 1,
                        embeddings_matrix.shape[1],
                        weights=[embeddings_matrix],
                        input_length=maxlen,
                        trainable=True)(inp)
        x_3 = SpatialDropout1D(0.2)(x_4)
        x_3 = encode(x_3)
        x_3 = Dropout(0.2)(x_3)
        x_3 = encode2(x_3)
        x_3 = Dropout(0.2)(x_3)
        x_3 = Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x_3)
        x_3 = Dropout(0.2)(x_3)
        avg_pool_3 = GlobalAveragePooling1D()(x_3)
        max_pool_3 = GlobalMaxPooling1D()(x_3)
        attention_3 = attention(x_3)
        x = keras.layers.concatenate([avg_pool_3, max_pool_3, attention_3])
        x = Dense(num_class, activation="sigmoid")(x)

        adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
        rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)
        model = Model(inputs=inp, outputs=x)
        model.compile(
            loss='binary_crossentropy',
            optimizer=rmsprop
            )
        return model




+ Train model and performance on validation dataset

In [ ]:
#train model using rcnn.  add fast-text pre-trained embedding.
from keras.backend.tensorflow_backend import set_session

#Set the GPU sources.
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

import random
random.seed = 42
import pandas as pd
from tensorflow import set_random_seed
set_random_seed(42)
from keras.preprocessing import text, sequence
from keras.callbacks import ModelCheckpoint, Callback
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from keras.layers import *
from classifier_rcnn import TextClassifier
from gensim.models.keyedvectors import KeyedVectors    #load word2vec model.
import pickle
import gc

# import os
# os.environ['OMP_NUM_THREADS'] = '8'  #set the number of threads


def getClassification(arr):
    arr = list(arr)
    res_values = []
    for i in range(20):                                            #the 20 classes.
        if arr[4 * i: 4 * (i+1)].index(max(arr[4 * i: 4 * (i+1)])) == 0:
            res_values.append(-2)
        elif arr[4 * i: 4 * (i+1)].index(max(arr[4 * i: 4 * (i+1)])) == 1:
            res_values.append(-1)
        elif arr[4 * i: 4 * (i+1)].index(max(arr[4 * i: 4 * (i+1)])) == 2:
            res_values.append(0)
        else:
            res_values.append(1)

    return res_values

class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
        self.val_accuracy = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = getClassification(self.model.predict(self.validation_data[0]))
        val_targ = getClassification(self.validation_data[1])
        #macro_F1: https://blog.csdn.net/sinat_28576553/article/details/80258619.   一般对于多分类任务来讲，macro 要优于 micro.
        _val_f1 = f1_score(val_targ, val_predict, average="macro")
        _val_recall = recall_score(val_targ, val_predict, average="macro")
        _val_precision = precision_score(val_targ, val_predict, average="macro")
        _val_accuracy = accuracy_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        self.val_accuracy.append(_val_accuracy)
        print(_val_f1, _val_precision, _val_recall, _val_accuracy)
        print("max f1: {}".format(max(self.val_f1s)))
        print("max acc: {}".format(max(self.val_accuracy)))
        return

import pandas as pd

data = pd.read_csv("preprocess/train_char.csv")
#eval() 函数用来执行一个字符串表达式，并返回表达式的值。 eg: eval('2+2') -> 4
data["content"] = [eval(i) for i in data.content]



validation = pd.read_csv("preprocess/validation_char.csv")
validation["content"] = [eval(i) for i in validation.content]

test = pd.read_csv('preprocess/test_char.csv')
test['content'] = [eval(i) for i in test.content]


EMBEDDING_FILE = '/media/chenshixin/My Passport/datasets/wiki.zh.vec'

model_dir = "train_rcnn/"
maxlen = 1000  
max_features = 20000
batch_size = 128     
epochs = 20
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(data.content.values) + list(validation.content.values) + list(test.content.values))   #Tokenizer().fit_on_texts(param:list)
X_train = tokenizer.texts_to_sequences(data.content.values)
X_valid = tokenizer.texts_to_sequences(validation.content.values)

input_train =  sequence.pad_sequences(X_train, maxlen=maxlen)  #Setting the same length of all the comment in train and validation dataset

input_validation = sequence.pad_sequences(X_valid, maxlen=maxlen)

def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
#Get a dict
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embeddings_matrix = np.zeros((nb_words + 1, embed_size))

for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)    #the function of embedding_index.get(word) is get the vector of the word.
    if embedding_vector is not None: embeddings_matrix[i] = embedding_vector
#for example, word_index:{'the':1}, embedding_index:{'the': vector}, embedding_matrix:{1: vector}

#line 86-128 one-hot enconding for all aspects in train and validation dataset

Y_train = data.iloc[:, 22:].values
Y_validation = validation.iloc[:, 22:].values

print("model_rcnn")
model = TextClassifier().model(embeddings_matrix, maxlen, word_index, 80)
file_path = model_dir + "model_rcnn_{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(file_path, verbose=1, save_weights_only=True)
metrics = Metrics()
callbacks_list = [checkpoint, metrics]
history = model.fit(input_train, Y_train, batch_size=batch_size, epochs=epochs,
                     validation_data=(input_validation, Y_validation), callbacks=callbacks_list, verbose=1)
del model1
del history
gc.collect()       #
K.clear_session()

#The basic usage of map see: https://www.runoob.com/python/python-func-map.html -> line 40.


#### Q41. 依据Kernel 中选择方法，对数据和代码进行改造，使其符合选择该问题。

回答：

#### Q42. 你现在的模型的准确率是多少？ 如何知道你的模型是不是真的学习了 而不是随机的进行猜测？

回答：

#### Q43. 你的模型现在准确度不高的原因，你猜测主要是什么？ 

回答：

#### Q43. 如前文所述，这个问题很难，其实现在也没有什么万灵药方法。 所以需要同学们多想想如何有效， 可以给大家参考的优化方式有， 修改vocabulary size, embedding size,去掉停用词，重新组合词组等。 并且结合使用LSTM， GRU， Bi-RNN， Stacked， Attentional, regularization, 等各种方法组合进行模型的优化， 至少进行10次优化，每次优化请按照以下步骤填写：

回答：

回答：

---这是一个实例----

第1次优化：

1. 存在的问题： loss下降太慢；
2. 准备进行的优化：减小模型的神经单元数量；
3. 期待的结果：loss下降加快；
4. 实际结果：loss下降的确加快(或者并没有加快)
5. 原因分析：模型神经元数量减小，收敛需要的次数减少，loss下降加快


---你的实验优化结构记录在此---

**第1次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第2次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第3次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第4次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第5次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第6次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第7次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第9次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第10次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：


## 最后一步： 使用Flask、Bottle、Bootstrap变成一个网络应用并且部署在服务器上，这样别人就可以直接通过网址访问你的应用啦。

最后一步，我们使用Bottle，Bootstrap,Flask等工具进行可视化现实，做出网页能够访问的形式，就像我们的第一个项目一样 😁.

## 本次项目的总结

请写项目的总结报告，描述此次项目的主要过程，其中遇到的问题，以及如何解决这些问题的，以及有什么经验和收获。

恭喜你，你完成了一个**十分**复杂的问题， 能完成这个问题，求是求是，你的能力其实已经达到了国内绝大多数公司的要求，你缺的只是熟练程度。 多多在 Kaggle， 阿里天池里边找一些自己感兴趣的问题，多练习练习。 熟能生巧。